In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

In [48]:
def build_lp_matrices(costs, supply, demand):
    m, n = costs.shape
    c = costs.flatten().astype(float)
    A_rows = []
    b = []

    # Oferta: sum_j x_ij = u_i
    for i in range(m):
        row = np.zeros(m*n)
        for j in range(n):
            row[i*n + j] = 1.0
        A_rows.append(row)
        b.append(float(supply[i]))

    # Demanda: sum_i x_ij = v_j
    for j in range(n):
        row = np.zeros(m*n)
        for i in range(m):
            row[i*n + j] = 1.0
        A_rows.append(row)
        b.append(float(demand[j]))

    A_eq = np.vstack(A_rows)
    b_eq = np.array(b, dtype=float)
    return c, A_eq, b_eq
    

In [49]:
# ---------- Utilidades ----------
def balance_problem(costs, supply, demand):
    s, d = supply.sum(), demand.sum()
    if abs(s - d) < 1e-12:
        return costs.copy(), supply.copy(), demand.copy(), False, False
    if s > d:  # falta demanda -> columna dummy
        extra = s - d
        costs2 = np.hstack([costs, np.zeros((costs.shape[0], 1))])
        demand2 = np.append(demand, extra)
        return costs2, supply.copy(), demand2, False, True
    else:      # falta oferta -> fila dummy
        extra = d - s
        costs2 = np.vstack([costs, np.zeros((1, costs.shape[1]))])
        supply2 = np.append(supply, extra)
        return costs2, supply2, demand.copy(), True, False

def basis_positions(X, eps=1e-12):
    return [(i, j) for i in range(X.shape[0]) for j in range(X.shape[1]) if X[i, j] > eps]


In [50]:
# ---------- Inicial: Costo mínimo ----------
def least_cost_initial(costs, supply, demand):
    m, n = costs.shape
    sup = supply.astype(float).copy()
    dem = demand.astype(float).copy()
    X = np.zeros((m, n), dtype=float)
    remaining = {(i, j) for i in range(m) for j in range(n)}

    while sup.sum() > 1e-12 and dem.sum() > 1e-12:
        best, best_cell = None, None
        for (i, j) in remaining:
            if sup[i] > 1e-12 and dem[j] > 1e-12:
                c = costs[i, j]
                if best is None or c < best or (c == best and (i, j) < best_cell):
                    best, best_cell = c, (i, j)

        if best_cell is None:
            # fallback defensivo
            for i in range(m):
                for j in range(n):
                    if sup[i] > 1e-12 and dem[j] > 1e-12:
                        best_cell = (i, j); break
                if best_cell: break
            if best_cell is None:
                return X  # por si ya está todo asignado

        i, j = best_cell
        q = min(sup[i], dem[j])
        X[i, j] = q
        sup[i] -= q
        dem[j] -= q

        # cerrar SOLO una dimensión cuando empatan
        if sup[i] <= 1e-12 and dem[j] <= 1e-12:
            for ii in range(m): remaining.discard((ii, j))     # cerrar columna
        elif sup[i] <= 1e-12:
            for jj in range(n): remaining.discard((i, jj))     # cerrar fila
        elif dem[j] <= 1e-12:
            for ii in range(m): remaining.discard((ii, j))     # cerrar columna

    return X

In [51]:
# ---------- MODI (u-v) ----------
def compute_potentials(costs, X):
    """
    Resuelve u_i + v_j = c_ij para celdas básicas usando mínimos cuadrados,
    con ancla u_0 = 0 (robusto a degeneración).
    """
    m, n = costs.shape
    basics = basis_positions(X)

    var_idx = {}
    k = 0
    for i in range(1, m):        # U[1..m-1]
        var_idx[('u', i)] = k; k += 1
    for j in range(n):           # V[0..n-1]
        var_idx[('v', j)] = k; k += 1

    A = []
    b = []
    for (i, j) in basics:
        row = np.zeros(k)
        if i != 0:
            row[var_idx[('u', i)]] = 1.0
        row[var_idx[('v', j)]] = 1.0
        A.append(row)
        b.append(costs[i, j])

    if len(A) == 0:
        # caso patológico (sin básicos) → devuelve ceros
        return np.zeros(m), np.zeros(n)

    A = np.vstack(A)
    b = np.array(b)
    sol, *_ = np.linalg.lstsq(A, b, rcond=None)

    U = np.zeros(m); V = np.zeros(n)
    for i in range(1, m):
        U[i] = sol[var_idx[('u', i)]]
    for j in range(n):
        V[j] = sol[var_idx[('v', j)]]
    return U, V

def reduced_costs(costs, U, V):
    return costs - (U[:, None] + V[None, :])


In [52]:

# ---------- Ciclo cerrado ----------
def _build_maps(basics):
    row_map, col_map = {}, {}
    for (i, j) in basics:
        row_map.setdefault(i, []).append((i, j))
        col_map.setdefault(j, []).append((i, j))
    return row_map, col_map

def find_cycle(X, start):
    basics = set(basis_positions(X))
    i0, j0 = start
    row_map, col_map = _build_maps(basics)
    row_map.setdefault(i0, []).append(start)
    col_map.setdefault(j0, []).append(start)

    def dfs(path, used, move_row):
        i, j = path[-1]
        nbrs = row_map.get(i, []) if move_row else col_map.get(j, [])
        for nxt in nbrs:
            if nxt == (i, j):
                continue
            if nxt == start and len(path) >= 4:
                return path + [start]
            if nxt in used:
                continue
            res = dfs(path + [nxt], used | {nxt}, not move_row)
            if res is not None:
                return res
        return None

    return dfs([start], set(), True) or dfs([start], set(), False)

def adjust_allocation(X, cycle, tol=1e-12):
    minus_cells = cycle[1::2]               # posiciones con signo '-'
    theta = min(X[i, j] for (i, j) in minus_cells)
    if theta <= tol:
        return X, theta
    for k, (i, j) in enumerate(cycle[:-1]): # último repite inicio
        if k % 2 == 0:  # '+'
            X[i, j] += theta
        else:           # '-'
            X[i, j] -= theta
    return X, theta


In [53]:

# ---------- Solver principal ----------
def transport_min_cost(costs, supply, demand, init="least_cost", max_iter=1000, tol=1e-9):
    costs2, supply2, demand2, added_row, added_col = balance_problem(costs, supply, demand)

    if init == "least_cost":
        X = least_cost_initial(costs2, supply2, demand2)
    elif init == "northwest":
        # versión noroeste por si quieres fallback
        m, n = costs2.shape
        X = np.zeros_like(costs2, dtype=float)
        i = j = 0
        sup, dem = supply2.copy(), demand2.copy()
        while i < m and j < n:
            q = min(sup[i], dem[j])
            X[i, j] = q
            sup[i] -= q; dem[j] -= q
            # tie-break: avanza columna si ambos llegan a 0
            if sup[i] <= 1e-12 and dem[j] <= 1e-12:
                j += 1
            elif sup[i] <= 1e-12:
                i += 1
            else:
                j += 1
    else:
        raise ValueError("init debe ser 'least_cost' o 'northwest'.")

    # <<< Guardia para evitar tu error >>>
    if X is None or not isinstance(X, np.ndarray):
        raise ValueError("La inicialización devolvió None. Revisa que least_cost_initial esté ejecutada y retorne X.")

    # … (resto de MODI igual)
    U, V = compute_potentials(costs2, X)
    # etc.

def reporte_df(X, supply, demand, origins=None, dests=None):
    m, n = X.shape
    if origins is None: origins = [f"O{i+1}" for i in range(m)]
    if dests   is None: dests   = [f"D{j+1}" for j in range(n)]
    df = pd.DataFrame(X, index=origins, columns=dests).astype(int)
    df["Oferta"] = supply.astype(int)
    total = list(demand.astype(int)) + [int(demand.sum())]
    df.loc["Demanda"] = total
    return df

In [54]:
# ---------- Solver principal (siempre retorna (X, z)) ----------
def transport_min_cost(costs, supply, demand, init="least_cost", max_iter=1000, tol=1e-9):
    costs2, supply2, demand2, added_row, added_col = balance_problem(costs, supply, demand)

    # Inicial factible
    if init == "least_cost":
        X = least_cost_initial(costs2, supply2, demand2)
    elif init == "northwest":
        m, n = costs2.shape
        X = np.zeros_like(costs2, dtype=float)
        i = j = 0
        sup, dem = supply2.copy(), demand2.copy()
        while i < m and j < n:
            q = min(sup[i], dem[j])
            X[i, j] = q
            sup[i] -= q; dem[j] -= q
            if sup[i] <= 1e-12 and dem[j] <= 1e-12:
                j += 1
            elif sup[i] <= 1e-12:
                i += 1
            else:
                j += 1
    else:
        raise ValueError("init debe ser 'least_cost' o 'northwest'.")

    # Guardias para NO retornar None
    if X is None or not isinstance(X, np.ndarray):
        raise ValueError("La inicialización devolvió None (X). Revisa least_cost_initial/northwest.")
    if X.shape != costs2.shape:
        raise ValueError("X tiene forma inconsistente con la matriz de costos balanceada.")

    # Iteraciones MODI
    for _ in range(max_iter):
        U, V = compute_potentials(costs2, X)
        D = reduced_costs(costs2, U, V)

        mask_nonbasic = X <= 1e-12
        min_rc = np.min(D[mask_nonbasic])
        if min_rc >= -tol:
            break  # óptimo alcanzado

        # Entrante: la con costo reducido más negativo
        i_in, j_in = np.where((D == min_rc) & mask_nonbasic)
        start = (int(i_in[0]), int(j_in[0]))
        cycle = find_cycle(X, start)
        if cycle is None:
            # Si no hay ciclo (muy raro/degenerado), salimos y devolvemos actual
            break
        X, theta = adjust_allocation(X, cycle, tol=tol)
        if theta <= tol:
            # Degeneración → no avanzamos; salimos con la mejor actual
            break

    # Quitar dummies si se agregaron
    if added_row:
        X = X[:-1, :]
    if added_col:
        X = X[:, :-1]

    # Costo con costos originales (sin dummies)
    z = float((X * costs).sum())
    return X, z


In [55]:
# ---------- Reporte ----------
def reporte_df(X, supply, demand, origins=None, dests=None):
    m, n = X.shape
    if origins is None: origins = [f"O{i+1}" for i in range(m)]
    if dests   is None: dests   = [f"D{j+1}" for j in range(n)]
    df = pd.DataFrame(X, index=origins, columns=dests).astype(int)
    df["Oferta"] = supply.astype(int)
    total = list(demand.astype(int)) + [int(demand.sum())]
    df.loc["Demanda"] = total
    return df

In [56]:
# ---------- Ejemplo con tus datos ----------
C = np.array([
    [120, 130,  41,  62],  # A
    [ 62,  40, 100, 110],  # B
    [102,  90, 122,  42],  # C
], dtype=float)
u = np.array([500, 700, 800], dtype=float)          # ofertas (A,B,C)
v = np.array([500, 400, 200, 900], dtype=float)     # demandas (1..4)

# 1) Matricial (PDF)
c, Aeq, beq = build_lp_matrices(C, u, v)
print("c^T (costos aplanados):", c)
print("A_eq shape:", Aeq.shape, "  b_eq shape:", beq.shape)

# 2) Resolver
Xopt, zopt = transport_min_cost(C, u, v, init="least_cost")
print("\nCosto mínimo:", int(zopt))
print(reporte_df(Xopt, u, v, origins=["A","B","C"], dests=["1","2","3","4"]))


c^T (costos aplanados): [120. 130.  41.  62.  62.  40. 100. 110. 102.  90. 122.  42.]
A_eq shape: (7, 12)   b_eq shape: (7,)

Costo mínimo: 106600
           1    2    3    4  Oferta
A        200    0  200  100     500
B        300  400    0    0     700
C          0    0    0  800     800
Demanda  500  400  200  900    2000
